In [14]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats

## CUPED

**CUPED (control upper pre experimented data)** — методика, предложенная аналитической командой Microsoft.  
Используется Amazon, eBay, Facebook, Google, Microsoft, Yahoo, Zynga, Netflix.  

Используя CUPED, мы учитываем уже **существующие данные** о поведении пользователя до эксперимента, чтобы **снизить дисперсию метрики и повысить чувствительность**.  
👉 Это позволяет получить тот же уровень статистической мощности при уменьшении размера выборки почти в два раза.


Вместо исходной метрики вводим новую CUPED-метрику. Рассмотрим, как новая метрика приводит к снижению дисперсии.

> 💬 Метод имеет немного условий для его применения.  

**1. Допустим, у нас есть две случайных величины $Y$ и $X$.**  

$Х$ — это случайная величина до начала эксперимента (то, что $Х$ берется из данных до эксперимента важно, т.к. именно поэтому мы можем вычислить среднее от $Х$),   
$У$ — после запуска эксперимента. 
Также есть параметр $\theta$, с помощью которого мы можем ввести следующее распределение:  

$Y_{CUPED}=Y-\theta(X-\overline X)$


**2. Найдем дисперсию нашей метрики:**  

$var_{srs}(Y_{CUPED})=var_{srs}(Y)+\theta^2var_{srs}(X)-2\theta cov_{srs}(X,Y)$

Задача сводится к тому, что мы хотим снизить дисперсию нашего распределения $Y_{CUPED}$.   

**4. Подберем такой параметр $\theta$, чтобы дисперсия была минимальна.**    

💬 Дисперсия — это квадратичная функция относительно $\theta$ (то есть парабола, у которой ветви направлены вверх). Мы берем производную дисперсии по $\theta$ и приравниваем к нулю. Таким образом, мы получим значение $\theta_{optimal}$, при котором дисперсия будет минимальна (нижнюю точку параболы): 

$\theta_{optimal}=cov_{srs}(X,Y)/var_{srs}(X)$

**4. Подставим оптимальное значение $\theta$ в формулу дисперсии:**

$var_{srs}{(Y_{CUPED})}_{min}=var_{srs}(Y)(1-\rho^2),\;\rho=corr_{srs}(X,Y)$

👉 Множитель $(1-\rho^2)$ говорит о том, что чем сильнее корреляция между $Х$ и $У$ (то есть чем лучше исторические данные предсказывают будущие), тем больше слагаемое $\rho^2$ — тем сильнее удается снизить дисперсию в результате CUPED. 
___
Выше мы провели математические преобразования, используя лишь основы теории вероятности. Добавим контекста из аналитики.   
$Y$ — это наша основная выборка, которую мы получили по результатам эксперимента.  
$X$ — это некоторая выборка по тем же экспериментальным единицам (например, по тем же самым пользователям, которая была получена до эксперимента).    

При этом для того, чтобы методика работала, вообще не обязательно, чтобы $X$ и $Y$ были связаны.  
* Если они не связаны $(\rho=0)$, то метрика CUPED превращается в исходную метрику (имеет такую же дисперсию, то есть мы не получаем никакой пользы).  
* Чем больше $X$ и $Y$ связаны, тем больше снижается дисперсия. Даже если это разные показатели. Мы можем использовать (для $X$) в метрике CUPED как прошлые значения того же показателя $Y$, так и любой другой показатель. 

На практике получается хорошо связывать расходы пользователей или количество размещаемых объявлений из периода до с периодом после. Этот метод также даёт простор для формирования метрик на основе машинного обучения (выбирая $X$, мы по сути решаем задачу предсказания $Y$ на основе данных из предыдущих периодов).


💬 Если по элементам из $Y$ (например, пользователям), нет исторических данных в $X$, то мы просто ставим 0.

В тех случаях, когда мы вносим сильное изменение и исторические данные будут плохо предсказывать тестовые данные, в качестве $Х$ и $У$ берем именно совокупные данные теста и контроля. Чем сильнее изменение, тем меньше будет корреляция $Х$ и $У$, и тем меньше снижение дисперсии. 
Если изменения сильные, то **Т-тест** и на исходной выборке покажет значимые отличия. А вот для более слабых изменений мы можем применить Т-тест не напрямую, а на метрику CUPED. Поскольку изменение слабое, то корреляция $Х$ и $У$ будет присутствовать, что снизит дисперсию. При этом на выборке с меньшей дисперсией Т-тест будет иметь большую мощность (с большей вероятностью сможем найти значимое отличие).

>📌 Нужно сначала найти метрику, которая хорошо скоррелирована, а только потом применять метод CUPED.

Рассмотрим пример применения CUPED для реальных результатов AB-теста.

Загружаем данные эксперимента:

In [15]:
data_test=pd.read_csv('promt_test.csv')

In [16]:
data_test

,test_group,user_bucket,revenue,pre_revenue
0,control,0.0,1.541235e+06,2.431853e+06
1,control,1.0,1.540157e+06,2.567391e+06
2,control,2.0,1.658264e+06,2.568697e+06
3,control,3.0,2.007450e+06,2.962268e+06
4,control,4.0,1.976987e+06,2.910092e+06
...,...,...,...,...
145,new_lk_rec,45.0,1.991105e+06,3.182075e+06
146,new_lk_rec,46.0,1.858896e+06,2.726123e+06
147,new_lk_rec,47.0,1.695438e+06,2.564411e+06
148,new_lk_rec,48.0,1.710189e+06,3.066091e+06


In [17]:
data_test[data_test['test_group'] == 'new_lk_rec'].shape

(50, 4)

Считаем среднее предданных:

In [18]:
data_test['pre_revenue_tg_average']=data_test.pre_revenue.mean() 

Считаем множитель для CUPED: 

In [19]:
data_test.revenue.shape

(150,)

In [20]:
data_test.pre_revenue.shape

(150,)

In [21]:
teta = np.cov(data_test.revenue,data_test.pre_revenue, ddof=1)[1,0]/np.var(data_test.pre_revenue, ddof=1)

In [22]:
corr = np.corrcoef(data_test.revenue,data_test.pre_revenue)[1,0] 

Считаем CUPED-метрику:

In [23]:
data_test['cuped_revenue'] = data_test.revenue - teta * (data_test.pre_revenue - data_test.pre_revenue_tg_average)

Считаем дисперсию разности средних для обычной и CUPED-метрики.  
Оцениваем теоретическое значение для CUPED дисперсии и посчитанное на реальных данных.

In [24]:
std_total=(data_test.groupby(['test_group'])['revenue'].var(ddof=1)[0]/50+data_test.groupby(['test_group'])['revenue'].var(ddof=1)[1]/50)**0.5

In [25]:
std_cuped=std_total*(1-corr**2)**0.5

In [26]:
(data_test.groupby(['test_group'])['cuped_revenue'].var(ddof=1)[0]/50+data_test.groupby(['test_group'])['cuped_revenue'].var(ddof=1)[1]/50)**0.5

24041.778158869776

In [27]:
std_cuped

23539.84378394034

Дисперсия для CUPED-метрики похожа при теоретическом расчете и для реальных данных. А вот дисперсия обычной метрики значительно выше.

In [28]:
std_total

46315.23775108171

In [29]:
np.std(data_test['cuped_revenue'], ddof=1)

116898.28643690675

In [30]:
np.std(data_test['revenue'], ddof=1) / 50

4600.006676945849

Сравним результаты применения t-теста к обычной и CUPED-метрике.

In [27]:
stats.ttest_ind(data_test[(data_test.test_group=='new_lk')].revenue,data_test[(data_test.test_group=='control')].revenue,equal_var = False)

Ttest_indResult(statistic=-0.7629734809218813, pvalue=0.4474961962789541)

Результат t-теста к CUPED метрике:

In [28]:
stats.ttest_ind(data_test[(data_test.test_group=='new_lk')].cuped_revenue,data_test[(data_test.test_group=='control')].cuped_revenue,equal_var = False)


Ttest_indResult(statistic=-2.882124337332291, pvalue=0.004883185468346531)

Видно, что p-value значительно ниже 1%, а значит, мы отклонить гипотезу о равенстве средних этих выборок.

Рассмотрим другую группу new_lk_rec. Ее среднее немного больше, чем в контроле.

In [31]:
stats.ttest_ind(data_test[(data_test.test_group=='new_lk_rec')].revenue,data_test[(data_test.test_group=='control')].revenue,equal_var = False)

Ttest_indResult(statistic=0.8804806811048778, pvalue=0.38081302386744664)

In [32]:
stats.ttest_ind(data_test[(data_test.test_group=='new_lk_rec')].cuped_revenue,data_test[(data_test.test_group=='control')].cuped_revenue,equal_var = False)

Ttest_indResult(statistic=-1.5816595073366941, pvalue=0.11696897103008692)

☝️ Видно, что p-value здесь больше — значит, мы не отвергаем гипотезу о равенстве средних этих выборок.